# Data base processing

In [111]:
# !pip install python-docx

# !pip install openpyxl

# !pip install xlsxwriter

## Import and Cleaning

### Import packages

In [112]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import tempfile
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
from docx.shared import Pt
from docx.oxml import OxmlElement

### Import file 

In [113]:
# Selectam fisierul
file_name = input("Introduceti denumirea fisierului pentru procesare: ")
file_name1 = f"{file_name}.xlsx"


df_original = pd.read_excel(file_name1)

### Preliminary cleaning

In [114]:
# Selectam datele originale si redenumim coloanele
df_modified = df_original.copy()
df_modified.rename(columns={'SbjNum': 'id', 'Srvyr': 'operator'}, inplace=True)

# Doar pentru SUA
#/////////////////////////////////////////////////////////////////////////////////
# Replace values in the Q_3 column
df_modified['cat'] = df_modified['cat'].replace({
    'Vorbitori de limbă rusă cu vârsta 15-29 de ani.': 'Russian speakers 15-29 years.',
    'Vorbitori de limbă rusă cu vârsta 30-39 de ani.': 'Russian speakers 30-39 years.',
    'Vorbitori de limbă română cu vârsta 15-29 de ani': 'Romanian speakers 15-29 years.'
})


# Replace values in the Q_3 column
df_modified['m'] = df_modified['m'].replace({
    'Oraș': 'Urban',
    'Sat': 'Rural'
})
#////////////////////////////////////////////////////////////////////////////////

# Stergem coloanele inutile
columns_to_delete = ['Filter', 'Cancel', 'UsrUnq', 'Upload', 'SubjData', 'RvwTime', 'RvwComment', 'SrvyrComment',
                    'Complete', 'Test', 'StopQ', 'ParentID', 'UTCDiff', 'QAScore', 'FrScName', 'ExReNum', 'VStart',
                    'VEnd', 'RvwName', 'SbjNam']

columns_to_delete = [col for col in columns_to_delete if col in df_modified.columns]
df_modified.drop(columns=columns_to_delete, inplace=True, errors='ignore')

# Trim all columns in the DataFrame
df_modified = df_modified.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Stergem coloanele care au valori identice
df_modified = df_modified.loc[:, df_modified.apply(pd.Series.nunique) != 1]



### Age column

In [115]:
# Calculăm statistica tuturor coloanelor
summary_stats = df_original.describe()

# Identificăm potențialele coloane care conțin vârsta respondentilor în funcție de datele statistice ale datelor
potential_age_columns = []

for col in summary_stats.columns:
    # Verificăm dacă media și deviația standard se află în intervalul definit
    if 20 <= summary_stats.at['mean', col] <= 80 and 5 <= summary_stats.at['std', col] <= 20:
     potential_age_columns.append(col)

# Verificăm dacă există coloane potențiale pentru vârstă
if potential_age_columns:
    age_column = potential_age_columns[0]

    # Verificăm dacă coloana identificată există deja în dataframe-ul modificat
    if age_column in df_modified.columns:
        # Definim intervalurile de vârstă
        age_bins = [18, 29, 37, 47, 57, 66, float('inf')]
        age_labels = ['18-29', '30-37', '38-47','48-57','58-66', '67+']

        # Înlocuim coloana identificată cu coloana nou creată cu etichetele de vârstă
        df_modified[age_column] = pd.cut(df_modified[age_column], bins=age_bins, labels=age_labels, include_lowest=True)
        df_modified.rename(columns={age_column: 'v1'}, inplace=True)
    else:
        print(f"Column {age_column} not found in df_modified.")

        # Definim intervalurile de vârstă în funcție de coloana identificată
        # Definim intervalurile de vârstă
        age_bins = [18, 29, 37, 47, 57, 66, float('inf')]
        age_labels = ['18-29', '30-37', '38-47','48-57','58-66', '67+']

        # Înlocuim datele lipsă din coloana identificată a 'vârstei' cu mediana valorilor
        df_modified[age_column].fillna(df_modified[age_column].median(), inplace=True)

        # Aplicăm etichetele de vârstă în funcție de valorile prestabilite
        df_modified['v1'] = pd.cut(df_modified[age_column], bins=age_bins, labels=age_labels, include_lowest=True)
else:
    print("No potential age columns found.")



### Weights 

#### V1

In [116]:

# # Population proportions
# population_proportions = {
#     ('v1', '18-29', 's', 'Bărbat'): 7.832863787,
#     ('v1', '18-29', 's', 'Femeie'): 7.867712534,
#     ('v1', '30-37', 's', 'Bărbat'): 7.919225872,
#     ('v1', '30-37', 's', 'Femeie'): 8.121530954,
#     ('v1', '38-47', 's', 'Bărbat'): 8.867385321,
#     ('v1', '38-47', 's', 'Femeie'): 9.223521284,
#     ('v1', '48-57', 's', 'Bărbat'): 7.792088727,
#     ('v1', '48-57', 's', 'Femeie'): 8.517023714,
#     ('v1', '58-66', 's', 'Bărbat'): 7.499369379,
#     ('v1', '58-66', 's', 'Femeie'): 9.489901461,
#     ('v1', '67+', 's', 'Bărbat'): 6.210016381,
#     ('v1', '67+', 's', 'Femeie'): 10.65936059,
#     ('reg', '1', 'm', 'Urban'): 8.723901517,
#     ('reg', '1', 'm', 'Rural'): 17.34464925,
#     ('reg', '2', 'm', 'Urban'): 5.087108494,
#     ('reg', '2', 'm', 'Rural'): 21.82653781,
#     ('reg', '3', 'm', 'Urban'): 27.17595166,
#     ('reg', '3', 'm', 'Rural'): 2.408172077,
#     ('reg', '4', 'm', 'Urban'): 3.376669407,
#     ('reg', '4', 'm', 'Rural'): 9.572135693,
#     ('reg', '5', 'm', 'Urban'): 1.846969537,
#     ('reg', '5', 'm', 'Rural'): 2.637904558,
#     # ('etnia', 'Moldovenească'): 72.43012346,
#     ('etnia', 'Rusă'): 6,
#     ('etnia', 'Ucraineană'): 6.21,
#     # ('etnia', 'Română'): 8.069876543,
#     ('etnia', 'Găgăuză'): 4,
#     ('etnia', 'Bulgară'): 1.93,
#     ('etnia', 'Alta, specificați'): 1.36,
#     ('regstat', 'Anenii Noi'): 5.254821581,
#     ('regstat', 'Bălți'): 4.953227638,
#     ('regstat', 'Botanica'): 5.480617285,
#     ('regstat', 'Buiucani'): 5.0735467,
#     ('regstat', 'Cahul'): 5.0735467,
#     ('regstat', 'Călărași'): 4.901030983,
#     ('regstat', 'Căușeni'): 4.726086588,
#     ('regstat', 'Centru'): 4.9495249,
#     ('regstat', 'Cimișlia'): 4.9495249,
#     ('regstat', 'Ciocana'): 5.374941572,
#     ('regstat', 'Comrat'): 4.874196089,
#     ('regstat', 'Edineț'): 4.860101798,
#     ('regstat', 'Fălești'): 4.78134895,
#     ('regstat', 'Florești'): 4.953824853,
#     ('regstat', 'Hîncești'): 4.87383776,
#     ('regstat', 'Orhei'): 4.884985787,
#     ('regstat', 'Rîșcani-Chișinău'): 5.374941572,
#     ('regstat', 'Rîșcani-Nord'): 4.880526576,
#     ('regstat', 'Soroca'): 4.82004853,
#     ('regstat', 'Ungheni'): 4.959319238
# }

# # Define the function to update weights for combined categories
# def update_weights_for_combined_category(df, combined_categories, target_proportion, current_weights):
#     # Combine counts for similar categories
#     combined_sample_count = sum(df[category].value_counts(dropna=False).get(value, 0)
#                                 for category, value in combined_categories)

#     # Calculate combined sample proportion
#     combined_sample_proportion = combined_sample_count / len(df)

#     # Calculate adjustment factor for the combined category
#     adjustment_factor = (target_proportion / combined_sample_proportion 
#                          if combined_sample_proportion > 0 else 1)

#     # Apply the adjustment factor to the weights of the matching rows
#     for category, value in combined_categories:
#         current_weights = current_weights.where(df[category] != value, 
#                                                 current_weights * adjustment_factor)

#     return current_weights

# # Check if all required columns for sample proportions are present in the DataFrame
# required_columns = set([key[0] for key in population_proportions.keys()])
# missing_columns = required_columns - set(df_modified.columns)

# if missing_columns:
#     print("Warning: Missing required columns for weight calculation:", missing_columns)
# else:
#     # Proceed with weight calculation

#     # Initialize weights to 1 for each respondent
#     df_modified['Weight'] = 1

#     # Handle the combined category 'Moldovenească' and 'Română'
#     combined_categories = [('etnia', 'Moldovenească'), ('etnia', 'Română')]
#     target_combined_proportion = 80.5 / 100  # 80.5% as a decimal
#     df_modified['Weight'] = update_weights_for_combined_category(
#         df_modified,
#         combined_categories,
#         target_combined_proportion,
#         df_modified['Weight']
#     )

#     # Iterate over each demographic characteristic (including regstat) and update weights
#     for key_tuple, pop_prop in population_proportions.items():
#         category, value = key_tuple[:2]  # Extract category and value
#         df_modified['Weight'] = update_weights_for_combined_category(
#             df_modified,
#             [(category, value)],
#             pop_prop / 100,  # Convert percentage to proportion
#             df_modified['Weight']
#         )

#     # Normalize the weights after each step
#     df_modified['Weight'] /= df_modified['Weight'].mean()

#     # Apply the final capping rules
#     df_modified['Weight'] = np.clip(df_modified['Weight'], 0.5, 2.0)


#### V2

In [117]:


# # Population proportions
# population_proportions = {
    # ('v1', '18-29', 's', 'Bărbat'): 7.832863787,
    # ('v1', '18-29', 's', 'Femeie'): 7.867712534,
    # ('v1', '30-37', 's', 'Bărbat'): 7.919225872,
    # ('v1', '30-37', 's', 'Femeie'): 8.121530954,
    # ('v1', '38-47', 's', 'Bărbat'): 8.867385321,
    # ('v1', '38-47', 's', 'Femeie'): 9.223521284,
    # ('v1', '48-57', 's', 'Bărbat'): 7.792088727,
    # ('v1', '48-57', 's', 'Femeie'): 8.517023714,
    # ('v1', '58-66', 's', 'Bărbat'): 7.499369379,
    # ('v1', '58-66', 's', 'Femeie'): 9.489901461,
    # ('v1', '67+', 's', 'Bărbat'): 6.210016381,
    # ('v1', '67+', 's', 'Femeie'): 10.65936059,
    # ('reg', '1', 'm', 'Urban'): 8.723901517,
    # ('reg', '1', 'm', 'Rural'): 17.34464925,
    # ('reg', '2', 'm', 'Urban'): 5.087108494,
    # ('reg', '2', 'm', 'Rural'): 21.82653781,
    # ('reg', '3', 'm', 'Urban'): 27.17595166,
    # ('reg', '3', 'm', 'Rural'): 2.408172077,
    # ('reg', '4', 'm', 'Urban'): 3.376669407,
    # ('reg', '4', 'm', 'Rural'): 9.572135693,
    # ('reg', '5', 'm', 'Urban'): 1.846969537,
    # ('reg', '5', 'm', 'Rural'): 2.637904558,
    # ('etnia', 'Moldovenească'): 72.43012346,
    # ('etnia', 'Rusă'): 6,
    # ('etnia', 'Ucraineană'): 6.21,
    # ('etnia', 'Română'): 8.069876543,
    # ('etnia', 'Găgăuză'): 4,
    # ('etnia', 'Bulgară'): 1.93,
    # ('etnia', 'Alta, specificați'): 1.36,
#     ('regstat', 'Anenii Noi'): 5.254821581,
#     ('regstat', 'Bălți'): 4.953227638,
#     ('regstat', 'Botanica'): 5.480617285,
#     ('regstat', 'Buiucani'): 5.0735467,
#     ('regstat', 'Cahul'): 5.0735467,
#     ('regstat', 'Călărași'): 4.901030983,
#     ('regstat', 'Căușeni'): 4.726086588,
#     ('regstat', 'Centru'): 4.9495249,
#     ('regstat', 'Cimișlia'): 4.9495249,
#     ('regstat', 'Ciocana'): 5.374941572,
#     ('regstat', 'Comrat'): 4.874196089,
#     ('regstat', 'Edineț'): 4.860101798,
#     ('regstat', 'Fălești'): 4.78134895,
#     ('regstat', 'Florești'): 4.953824853,
#     ('regstat', 'Hîncești'): 4.87383776,
#     ('regstat', 'Orhei'): 4.884985787,
#     ('regstat', 'Rîșcani-Chișinău'): 5.374941572,
#     ('regstat', 'Rîșcani-Nord'): 4.880526576,
#     ('regstat', 'Soroca'): 4.82004853,
#     ('regstat', 'Ungheni'): 4.959319238
# }


# # Initialize weights to 1 for each respondent
# df_modified['Weight'] = 1

# # Define a tolerance level for convergence
# tolerance = 0.02

# # Pre-calculate filter conditions for each category-value pair
# filter_conditions = {}
# for key_tuple, pop_prop in population_proportions.items():
#     category_value_pairs = [(key_tuple[i], key_tuple[i+1]) for i in range(0, len(key_tuple), 2)]
#     filter_condition = pd.Series([True] * len(df_modified))
#     for category, value in category_value_pairs:
#         filter_condition &= (df_modified[category] == value)
#     filter_conditions[key_tuple] = filter_condition

# # Iterate until convergence
# converged = False
# while not converged:
#     converged = True
#     total_sample_count = df_modified['Weight'].sum()

#     for key_tuple, pop_prop in population_proportions.items():
#         filter_condition = filter_conditions[key_tuple]

#         # Calculate current sample proportion
#         sample_count = df_modified[filter_condition]['Weight'].sum()
#         sample_proportion = sample_count / total_sample_count

#         # Check if the current sample proportion is within the tolerance of the population proportion
#         if abs(sample_proportion - (pop_prop / 100)) > tolerance:
#             converged = False
#             adjustment_factor = (pop_prop / 100) / sample_proportion

#             # Apply the adjustment factor to the weights of the matching rows
#             df_modified.loc[filter_condition, 'Weight'] *= adjustment_factor

# # Normalize the weights after convergence
# df_modified['Weight'] /= df_modified['Weight'].mean()

# # Apply the final capping rules
# df_modified['Weight'] = np.clip(df_modified['Weight'], 0.5, 2.0)




In [118]:
# # Population proportions
# population_proportions = {
#     ('v1', '18-29', 's', 'Bărbat'): 7.832863787,
#     ('v1', '18-29', 's', 'Femeie'): 7.867712534,
#     ('v1', '30-37', 's', 'Bărbat'): 7.919225872,
#     ('v1', '30-37', 's', 'Femeie'): 8.121530954,
#     ('v1', '38-47', 's', 'Bărbat'): 8.867385321,
#     ('v1', '38-47', 's', 'Femeie'): 9.223521284,
#     ('v1', '48-57', 's', 'Bărbat'): 7.792088727,
#     ('v1', '48-57', 's', 'Femeie'): 8.517023714,
#     ('v1', '58-66', 's', 'Bărbat'): 7.499369379,
#     ('v1', '58-66', 's', 'Femeie'): 9.489901461,
#     ('v1', '67+', 's', 'Bărbat'): 6.210016381,
#     ('v1', '67+', 's', 'Femeie'): 10.65936059,
#     ('reg', '1', 'm', 'Urban'): 8.723901517,
#     ('reg', '1', 'm', 'Rural'): 17.34464925,
#     ('reg', '2', 'm', 'Urban'): 5.087108494,
#     ('reg', '2', 'm', 'Rural'): 21.82653781,
#     ('reg', '3', 'm', 'Urban'): 27.17595166,
#     ('reg', '3', 'm', 'Rural'): 2.408172077,
#     ('reg', '4', 'm', 'Urban'): 3.376669407,
#     ('reg', '4', 'm', 'Rural'): 9.572135693,
#     ('reg', '5', 'm', 'Urban'): 1.846969537,
#     ('reg', '5', 'm', 'Rural'): 2.637904558,
#     ('etnia', 'Moldovenească'): 72.43012346,
#     ('etnia', 'Rusă'): 6,
#     ('etnia', 'Ucraineană'): 6.21,
#     ('etnia', 'Română'): 8.069876543,
#     ('etnia', 'Găgăuză'): 4,
#     ('etnia', 'Bulgară'): 1.93,
#     ('etnia', 'Alta, specificați'): 1.36,
#     ('regstat', 'Anenii Noi'): 5.254821581,
#     ('regstat', 'Bălți'): 4.953227638,
#     ('regstat', 'Botanica'): 5.480617285,
#     ('regstat', 'Buiucani'): 5.0735467,
#     ('regstat', 'Cahul'): 5.0735467,
#     ('regstat', 'Călărași'): 4.901030983,
#     ('regstat', 'Căușeni'): 4.726086588,
#     ('regstat', 'Centru'): 4.9495249,
#     ('regstat', 'Cimișlia'): 4.9495249,
#     ('regstat', 'Ciocana'): 5.374941572,
#     ('regstat', 'Comrat'): 4.874196089,
#     ('regstat', 'Edineț'): 4.860101798,
#     ('regstat', 'Fălești'): 4.78134895,
#     ('regstat', 'Florești'): 4.953824853,
#     ('regstat', 'Hîncești'): 4.87383776,
#     ('regstat', 'Orhei'): 4.884985787,
#     ('regstat', 'Rîșcani-Chișinău'): 5.374941572,
#     ('regstat', 'Rîșcani-Nord'): 4.880526576,
#     ('regstat', 'Soroca'): 4.82004853,
#     ('regstat', 'Ungheni'): 4.959319238
# }


# # Convert population proportions to DataFrame for easier processing
# pop_prop_df = pd.DataFrame([(k[0], k[1], k[2], k[3], v) for k, v in population_proportions.items()], 
#                            columns=['Category1', 'Category2', 'Category3', 'Category4', 'PopProp'])
# # Initialize weights to 1 for each respondent
# df_modified['Weight'] = 1

# def adjust_weights(df, pop_prop_df, tolerance=0.09, max_iterations=1048, verbose=False):
#     # Precompute filtered DataFrames for each category-value pair
#     filtered_dfs = {}
#     for _, row in pop_prop_df.iterrows():
#         category, value = row['Category'], row['Value']
#         filtered_dfs[(category, value)] = df[df[category] == value]

#     current_iteration = 0
#     converged = False

#     while not converged and current_iteration < max_iterations:
#         converged = True
#         current_iteration += 1

#         total_sample_count = df['Weight'].sum()

#         for _, row in pop_prop_df.iterrows():
#             category, value, pop_prop = row['Category'], row['Value'], row['PopProp']
#             df_filtered = filtered_dfs[(category, value)]

#             sample_count = df_filtered['Weight'].sum()
#             sample_proportion = sample_count / total_sample_count if total_sample_count > 0 else 0

#             if abs(sample_proportion - (pop_prop / 100)) > tolerance:
#                 converged = False
#                 adjustment_factor = (pop_prop / 100) / sample_proportion if sample_proportion > 0 else 1
#                 df.loc[df[category] == value, 'Weight'] *= adjustment_factor

#                 if verbose:
#                     print(f"Iteration {current_iteration}, Adjusting {category}-{value}: {adjustment_factor}")

#         if df['Weight'].sum() > 0:
#             df['Weight'] /= df['Weight'].mean()

#     df['Weight'] = np.clip(df['Weight'], 0.5, 2.0)

#     if verbose:
#         print("Convergence Status:", "Converged" if converged else "Not Converged")
#         print("Total Iterations:", current_iteration)

#     return df

# # Now call the function with df_modified and pop_prop_df
# df_adjusted = adjust_weights(df_modified, pop_prop_df, tolerance=0.02, max_iterations=1048, verbose=True)


In [119]:
# def calculate_sample_proportions(df, pop_prop_df):
#     total_weight = df['Weight'].sum()
#     calculated_proportions = {}

#     for _, row in pop_prop_df.iterrows():
#         category, value = row['Category'], row['Value']
#         # Sum the weights for the specific category-value pair
#         sum_weights = df[df[category] == value]['Weight'].sum()

#         # Calculate the proportion
#         proportion = sum_weights / total_weight if total_weight > 0 else 0
#         calculated_proportions[(category, value)] = proportion

#     return calculated_proportions

# # Assuming df_modified is your dataset after applying weights
# calculated_proportions = calculate_sample_proportions(df_modified, pop_prop_df)

# # Print calculated proportions
# for category_value, proportion in calculated_proportions.items():
#     print(f"{category_value}: {proportion}")


### Dissagreagtion column sellection + 

In [120]:
# Correcting the name of the operator
df_modified['operator'] = df_modified['operator'].str.replace('.', ' ').apply(lambda x: ' '.join(x.capitalize() for x in x.split()))

# Interactive input for multiple additional column names
input_columns = input("Enter the names of the columns for disaggregation separated by a comma (leave blank if none): ")

# Process the input
if input_columns.strip():
    disaggregation_columns = [col.strip() for col in input_columns.split(',')]
else:
    disaggregation_columns = []  # No columns selected

# List of columns to select (columns starting with "Q_" or "T_", disaggregation columns)
selected_columns = [col for col in df_modified.columns if col.startswith(("Q_", "T_"))] + disaggregation_columns


# # Perform replacement of -1 with 'Necunoscut' only in disaggregation columns
# for col in disaggregation_columns:
#     if col in df_modified.columns:
#         df_modified[col] = df_modified[col].replace('-1', 'Necunoscut')


# Add 'Weight' column to the list if it exists in the DataFrame
if 'Weight' in df_modified.columns:
    selected_columns.append('Weight')

# Inlocuirea valorii -1 cu  None
df_modified.replace('-1', None, inplace=True)

# Create a new DataFrame with selected columns
df_selected = df_modified[selected_columns]

# coloane cu calcul special
suplimentar_columns = ['Q_47_O1', 'Q_47_O2', 'Q_47_O3','Q_26_O1','Q_26_O2','Q_26_O3']  


### Transformig  multiple columns to dummies

In [121]:
dummy_columns_list = []

for col in df_selected.filter(regex=r'^Q_\d+_O\d+$').columns:
    if col not in suplimentar_columns:
        # Extract question_number and option_number
        match = re.match(r'^Q_(\d+)_O(\d+)$', col)
        if match:
            question_number, option_number = map(int, match.groups())
            new_column_name = f"Q{question_number}_{option_number}"

            # Use pd.get_dummies directly on the string values
            dummy_columns = pd.get_dummies(df_selected[col], prefix=new_column_name, dummy_na=False)
            dummy_columns_list.append(dummy_columns)

# Concatenate all dummy columns
df_dummies = pd.concat(dummy_columns_list, axis=1)

# Create a list of columns to add to df_dummies, initially just the disaggregation columns
columns_to_add = disaggregation_columns

# Add 'Weight' column to the list if it exists in df_selected
if 'Weight' in df_selected.columns:
    columns_to_add.append('Weight')

# Add the selected columns to df_dummies
df_dummies = pd.concat([df_dummies, df_selected[columns_to_add]], axis=1)


# Concatenate df_dummies with df_modified
df_modified = pd.concat([df_modified, df_dummies], axis=1)


### Frequencies calculations and weights aplicationv + special q_option questions

In [122]:


def create_weighted_freq_table(df, response_column, disaggregation_columns, weight_column):
    # Calculate 'N' as unweighted count of responses
    basic_freq_table = df.groupby(response_column).size().rename('N').reset_index()

    # Calculate weighted total for the 'Total' column
    weighted_total = df.groupby(response_column)[weight_column].sum().rename('Weighted_Total').reset_index()
    total_weight = weighted_total['Weighted_Total'].sum()
    weighted_total['Total'] = (weighted_total['Weighted_Total'] / total_weight) * 100
    weighted_total['Total'] = weighted_total['Total'].round(1)

    # Merge the unweighted 'N' with the weighted 'Total'
    merged_table = basic_freq_table.merge(weighted_total[[response_column, 'Total']], on=response_column)

    pivot_tables = [merged_table.set_index(response_column)]
    for disaggregation_column in disaggregation_columns:
        # For disaggregated columns, use weighted sum
        weighted_counts = df.groupby([response_column, disaggregation_column])[weight_column].sum().reset_index(name='Weighted_N')
        pivot_table = weighted_counts.pivot(index=response_column, columns=disaggregation_column, values='Weighted_N').fillna(0)
        pivot_tables.append(pivot_table)

    return pd.concat(pivot_tables, axis=1).reset_index()



def create_unweighted_freq_table(df, response_column, disaggregation_columns):
    # Function to create frequency table without weights (count)
    basic_freq_table = df.groupby(response_column).size().rename('N').reset_index()
    basic_freq_table['Total'] = (basic_freq_table['N'] / basic_freq_table['N'].sum()) * 100
    basic_freq_table['Total'] = basic_freq_table['Total'].round(1)

    pivot_tables = [basic_freq_table.set_index(response_column)]
    for disaggregation_column in disaggregation_columns:
        counts = df.groupby([response_column, disaggregation_column]).size().reset_index(name='N')
        pivot_table = counts.pivot(index=response_column, columns=disaggregation_column, values='N').fillna(0)
        pivot_tables.append(pivot_table)

    return pd.concat(pivot_tables, axis=1).reset_index()


apply_weights = 'n'  # Default to not applying weights

# Check if 'Weight' column is present in the DataFrame
if 'Weight' in df_modified.columns:
    apply_weights = input("Apply weights? (y/n): ").strip().lower()
   

# Define regular expressions for each category
q_digit_regex = re.compile(r'^Q_\d+$')
t_q_regex = re.compile(r'^T_Q_\d+_\d+$')

# Filter columns based on regular expressions
q_digit_columns = [col for col in df_selected.columns if q_digit_regex.match(col)]
t_q_columns = [col for col in df_selected.columns if t_q_regex.match(col)]



# Extend the q_digit_columns list with additional columns
q_digit_columns.extend(suplimentar_columns)

# Create frequency tables based on user choice
all_separated_tables = {}
for question_column in q_digit_columns + t_q_columns:
    # Ensure 'Weight' is not treated as a disaggregation column
    disaggregation_columns_excluding_weight = [col for col in disaggregation_columns if col != 'Weight']

    if apply_weights == 'y':
        separated_table = create_weighted_freq_table(df_selected, question_column, disaggregation_columns_excluding_weight, 'Weight')
    else:
        separated_table = create_unweighted_freq_table(df_selected, question_column, disaggregation_columns_excluding_weight)
    all_separated_tables[question_column] = separated_table

# Export frequency tables to Excel in separate sheets
with pd.ExcelWriter('frequency_tables.xlsx', engine='xlsxwriter') as writer:
    # Export q_digit_tables to one sheet
    startrow_q_digit = 1
    for question_column in q_digit_columns:
        table = all_separated_tables[question_column]
        if isinstance(table.columns, pd.MultiIndex):
            table.columns = ['_'.join(map(str, col_tuple)) for col_tuple in table.columns.values]
        table.to_excel(writer, sheet_name='Q_digit_Tables', index=False, startrow=startrow_q_digit)
        startrow_q_digit += len(table) + 3  # Adjust the start row for the next table

    # Export t_q_tables to another sheet
    startrow_t_q = 1
    for question_column in t_q_columns:
        table = all_separated_tables[question_column]
        if isinstance(table.columns, pd.MultiIndex):
            table.columns = ['_'.join(map(str, col_tuple)) for col_tuple in table.columns.values]
        table.to_excel(writer, sheet_name='T_Q_Tables', index=False, startrow=startrow_t_q)
        startrow_t_q += len(table) + 3  # Adjust the start row for the next table

    # Add additional code for df_dummies if needed

    df_dummies.to_excel(writer, sheet_name='dummies_+variabila', index=False)
    

### Questions selection 

In [123]:
# Selectam fisierul
filename = input("Introduceti denumirea fisierului care contine textul intrebarilor: ")
filename1 = f"{filename}.xls"

df_questions = pd.read_excel(filename1)

# Method 1: Using loc
questions_df_filtered = df_questions.loc[df_questions['Is Topic'] != False]
# Dropping the first two rows of questions_df_filtered
questions_df_filtered = questions_df_filtered.iloc[2:]
# Dropping rows where 'Question Index' column has NA/null values
questions_df_filtered = questions_df_filtered.dropna(subset=['Question Index'])
# Selecting only 'Question Index' and 'Text' columns
questions_df_filtered = questions_df_filtered[['Question Index', 'Text']]
# Replacing <b> and </b> with an empty string in the 'Text' column
questions_df_filtered['Text'] = questions_df_filtered['Text'].replace('<b>|</b>', '', regex=True)
# Extracting column names that start with "Q" and "T_Q"
filtered_columns = [col for col in df_original.columns if col.startswith('Q_') or col.startswith('T_Q')]

# Creating a new DataFrame with these column names as values in a single column
df_columns = pd.DataFrame({'column_names': filtered_columns})

# Create a new DataFrame for modified data with T_Q_digit part as the header of each section
df_columns_new = pd.DataFrame(columns=df_columns.columns)

# Initialize last_group as None to indicate no group has been encountered yet
last_group = None

for index, row in df_columns.iterrows():
    # Extract the group number from the 'T_Q' pattern
    match = re.match(r'T_Q_(\d+)_\d+', str(row['column_names']))
    if match:
        current_group = int(match.group(1))

        # Insert a row with the T_Q_digit part if it's the first group or if the group number changes
        if last_group is None or current_group != last_group:
            df_columns_new = df_columns_new.append(pd.Series({'column_names': f'T_Q_{current_group}'}), ignore_index=True)

        # Update the last seen group number
        last_group = current_group

    # Append the current row
    df_columns_new = df_columns_new.append(row)

# Reset index for the new DataFrame
df_columns_new.reset_index(drop=True, inplace=True)

def filter_specific_pattern(dataframe, pattern_to_keep, general_pattern):
    # Function to check if the entry matches the general pattern and ends with the specific pattern
    def entry_matches(entry):
        return re.match(general_pattern, entry) and entry.endswith(pattern_to_keep)

    # Apply the filter to the DataFrame
    filtered_dataframe = dataframe[
        dataframe['column_names'].apply(lambda x: not re.match(general_pattern, x) or entry_matches(x))
    ]

    return filtered_dataframe

# The specific pattern we want to keep
specific_ending = '_O1'

# The general pattern that the questions must match (e.g., 'Q_number_Osomething')
question_pattern = r'Q_\d+_O\d+'

# Apply the filter function to the DataFrame
df_columns_new = filter_specific_pattern(df_columns_new, specific_ending, question_pattern)

# Step 1: Remove '_O1' from the end of strings in 'column_names'
df_columns_new['column_names'] = df_columns_new['column_names'].str.replace('_O1', '', regex=False)

# Step 2: Drop rows where 'column_names' ends with 'S'
df_columns_new = df_columns_new[~df_columns_new['column_names'].str.endswith('S')]

# Truncate questions_df_filtered to match the length of df_columns_new
questions_df_filtered = questions_df_filtered.iloc[:len(df_columns_new)]

# Assign values from 'column_names' in df_columns_new to 'q_index' in questions_df_filtered
questions_df_filtered['q_index'] = df_columns_new['column_names'].values

# Reorder columns and drop 'Question index'
questions_df_filtered = questions_df_filtered[['q_index', 'Text']].copy()

questions_filtered_df = questions_df_filtered

## Dummies Processing

In [124]:
def sum_and_rename(df, disaggregation_columns, weight_column='Weight', apply_weights='n'):
    result_df = pd.DataFrame()

    # Check if weight column is present and user chose to apply weights
    weight_present = weight_column in df.columns and apply_weights == 'y'

    for column in disaggregation_columns:
        if column in df.columns:
            result_df[column] = df[column]

    for column in df.columns:
        if column.startswith('Q') and column.count('_') == 2:
            if weight_present:
                weighted_column = df[column] * df[weight_column]
            else:
                weighted_column = df[column]

            question, option, response = column.split('_')
            result_column = f'{question}_{response}'

            if result_column in result_df.columns:
                result_df[result_column] += weighted_column
            else:
                new_df = pd.DataFrame({result_column: weighted_column})
                result_df = pd.concat([result_df, new_df], axis=1)

    return result_df


# Usage of the function
result_df = sum_and_rename(df_dummies.copy(), disaggregation_columns, 'Weight', apply_weights)

# Step 1: Extract Question and Response
question_response_data = []
for column in result_df.columns:
    if column.startswith('Q'):
        question, response = column.split('_', 1)
        question_response_data.append({
            'q': question,
            'Răspunsuri': response,
            'Column': column
        })

# Convert to DataFrame
question_response_df = pd.DataFrame(question_response_data)


# Determine if weights should be applied
weight_present = 'Weight' in df_dummies.columns and apply_weights == 'y'

# Step 2 and 3: Calculate Absolute and Relative Frequency
frequency_data = []
for _, row in question_response_df.iterrows():
    column = row['Column']

    # Calculate absolute frequency as a count of non-zero responses (unweighted)
    abs_frequency = (result_df[column] != 0).sum()

    # Calculate the total responses for the question (considering multiple selections)
    if weight_present:
        # Use the weighted sum for the total responses across all options for the question
        total_responses = result_df[[col for col in result_df.columns if col.startswith(row['q'])]].multiply(result_df['Weight'], axis=0).sum().sum()
    else:
        # Use the sum of all responses for all options within the question (unweighted)
        total_responses = result_df[[col for col in result_df.columns if col.startswith(row['q'])]].sum().sum()

    # Calculate relative frequency considering multiple selections
    if weight_present:
        rel_frequency = (result_df[column].multiply(result_df['Weight']).sum() / total_responses * 100) if total_responses != 0 else 0
    else:
        # For the unweighted scenario, calculate the relative frequency normally
        rel_frequency = (result_df[column].sum() / total_responses * 100) if total_responses != 0 else 0

    frequency_data.append({
        'q': row['q'],
        'Răspunsuri': row['Răspunsuri'],
        'N': abs_frequency,  # Unweighted count of responses
        'Total': rel_frequency  # Weighted or unweighted percentage
    })

# Convert to DataFrame
frequency_df = pd.DataFrame(frequency_data)

# Step 4: Calculate Relative Frequencies for Disaggregation Categories
disaggregation_columns = [col for col in disaggregation_columns if col != 'Weight']
for col in disaggregation_columns:
    for category in result_df[col].unique():
        category_filter = result_df[col] == category
        category_count = result_df[category_filter].shape[0]  # Count of entries in the category

        for _, row in question_response_df.iterrows():
            question_col = row['Column']

            # Calculate absolute frequency for the category
            abs_frequency = result_df[category_filter][question_col].sum()

            # Calculate and store relative frequency
            rel_frequency = (abs_frequency / category_count * 100) if category_count != 0 else 0
            frequency_df.loc[(frequency_df['q'] == row['q']) & (frequency_df['Răspunsuri'] == row['Răspunsuri']), f'{category}'] = rel_frequency

# Step 5: Assemble the Frequency Table
frequency_df = frequency_df.round(1)  # Round the frequencies for better readability


# Sorting values within each 'q' group based on the 'N' column in descending order
frequency_df = frequency_df.groupby('q', group_keys=False).apply(lambda x: x.sort_values('N', ascending=False))

# Resetting the index after sorting
frequency_df = frequency_df.reset_index(drop=True)

results_multiple = frequency_df.copy()


## Frequency tables processing 

In [125]:
df_q = pd.read_excel('frequency_tables.xlsx', sheet_name='Q_digit_Tables')
df_t = pd.read_excel('frequency_tables.xlsx', sheet_name='T_Q_Tables')


# Assuming df_q is your DataFrame
new_column_names = ['Răspunsuri'] + list(df_q.iloc[0, 1:])
df_q.columns = new_column_names
new_column_names = ['Răspunsuri'] + list(df_t.iloc[0, 1:])
df_t.columns = new_column_names

#### Q_digit Tables

In [126]:
# POCESAREA INTREBARILOR IN FORMAT Q_DIGIT

#//////////////////////////////////////////////////////////////////////////////////////////

# Assuming df_q is your DataFrame

# Drop rows where 'question' is NaN
df_q = df_q.dropna(subset=['Răspunsuri'])

# Calculul frecventelor relative pentru coloanele dezagregate

# Identify the rows where a new question starts
question_indices = df_q[df_q['Răspunsuri'].str.startswith('Q_')].index

# Iterate through each question section
for i in range(len(question_indices) - 1):
    start_idx = question_indices[i]
    end_idx = question_indices[i + 1]

    # Extract the relevant section of the DataFrame
    section = df_q.loc[start_idx+1:end_idx]

    # Convert columns to numeric (excluding the first 3 columns)
    section.iloc[:, 3:] = section.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')

    # Calculate the sum of each column in the section (excluding the first 3 columns)
    col_sums = section.iloc[:, 3:].sum(axis=0)

    # Avoid division by zero
    col_sums_nonzero = col_sums.replace(0, 1)  # Replace zeros with 1 to avoid division by zero
    section.iloc[:, 3:] = section.iloc[:, 3:].div(col_sums_nonzero, axis=1) * 100

    # Replace NaN values with 0 after division
    section.iloc[:, 3:].fillna(0, inplace=True)

    # Update the original DataFrame with the calculated values
    df_q.loc[start_idx+1:end_idx] = section

# Process the last section separately
start_idx = question_indices[-1]
section = df_q.loc[start_idx+1:]

# Repeat the normalization steps for the last section
section.iloc[:, 3:] = section.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')
col_sums = section.iloc[:, 3:].sum(axis=0)
col_sums_nonzero = col_sums.replace(0, 1)
section.iloc[:, 3:] = section.iloc[:, 3:].div(col_sums_nonzero, axis=1) * 100
section.iloc[:, 3:].fillna(0, inplace=True)
df_q.loc[start_idx+1:] = section



# Assuming df_q is your DataFrame
# Find the index of the "Relative Frequency" column
relative_frequency_index = df_q.columns.get_loc("Total")

# Select columns after the "Relative Frequency" column
columns_to_round = df_q.columns[relative_frequency_index + 1:]

# Iterate through the columns, convert to numeric, and round
for column in columns_to_round:
    # Try to convert the column to numeric
    df_q[column] = pd.to_numeric(df_q[column], errors='coerce')

    # Round the numeric values in the column
    df_q[column] = df_q[column].round(1)

# Reset the index of the DataFrame
df_q.reset_index(drop=True, inplace=True)

# Create a new column named 'q' as the first column
df_q.insert(0, 'q', '')

# Iterate through the DataFrame to find rows that start with 'Q'
for index, row in df_q.iterrows():
    if row['Răspunsuri'].startswith('Q'):
        # Copy the value in the corresponding row to the 'q' column
        df_q.at[index, 'q'] = row['Răspunsuri']

# Forward fill values in the 'q' column
df_q['q'] = df_q['q'].replace('', method='ffill')

# Drop rows where 'question' column starts with 'Q'
df_q = df_q[~df_q['Răspunsuri'].str.startswith('Q')]

# Reset the index after dropping rows
df_q = df_q.reset_index(drop=True)

# Assuming df_q is your DataFrame
# Create a new DataFrame to store the results
df_q_result = pd.DataFrame(columns=df_q.columns)

# Iterate through each unique question in the 'q' column
for question in df_q['q'].unique():
    # Filter the DataFrame for the current question
    df_subset = df_q[df_q['q'] == question]

    # Calculate the total for each column and insert it as a new row
    total_row = df_subset.sum(axis=0)
    total_row['Răspunsuri'] = "Total"

    # Append the total row to the result DataFrame
    df_q_result = pd.concat([df_q_result, df_subset, pd.DataFrame([total_row])], ignore_index=True)

# Reset the index of the result DataFrame
df_q_result.reset_index(drop=True, inplace=True)

# Convert 'Total' and 'Relative Frequency' columns to numeric
df_q['N'] = pd.to_numeric(df_q['N'], errors='coerce')
df_q['Total'] = pd.to_numeric(df_q['Total'], errors='coerce')


def adjust_total_column(df, target_sum=100):
    # Create a copy of the DataFrame to avoid modifying the original
    adjusted_df = df.copy()

    # Identify the index of the 'Total' column
    total_column_index = adjusted_df.columns.get_loc('Total')

    # Iterate through all columns including 'Total'
    for column_index in range(total_column_index, len(adjusted_df.columns)):
        column_name = adjusted_df.columns[column_index]

        # Dictionary to track whether the adjustment has been applied for each question
        adjusted_dict = {}

        # Iterate through the unique questions
        for q in adjusted_df['q'].unique():
            # Fill NaN values with 0 for the current column and question
            adjusted_df[column_name].fillna(0, inplace=True)

            # Filter rows for the current question
            q_rows = adjusted_df[adjusted_df['q'] == q]

            # Identify the maximum value in the current column for the current question
            max_value = q_rows[column_name].max()

            # Check if the maximum value has already been adjusted for the current question
            if (q, column_name) not in adjusted_dict:
                # Calculate the sum for the current column and question
                current_sum = q_rows[column_name].sum()

                # Check if the sum is non-zero before making adjustments
                if current_sum != 0:
                    # Calculate the difference from the target sum
                    difference = target_sum - current_sum

                    # Adjust the maximum value based on whether the sum was less or more than the target sum
                    adjusted_df.loc[q_rows[column_name].idxmax(), column_name] += difference

                    # Mark that adjustment has been applied for this question and column
                    adjusted_dict[(q, column_name)] = True

    return adjusted_df

# Call the function to adjust only the first identified max value in each column to the right of and including 'Total'
df_q = adjust_total_column(df_q)


# SORTARE CU NU STIU/NU RASPUND
# Function to sort each group by 'Relative Frequency' in descending order placing 'Nu ştiu' and 'Nu răspund' at the bottom

def sort_within_group(group):
    non_nu_values = group.loc[group['Răspunsuri'].ne('Nu ştiu') & group['Răspunsuri'].ne('Nu răspund') & group['Răspunsuri'].ne('Total')]
    nu_values = group.loc[group['Răspunsuri'].isin(['Nu ştiu', 'Nu răspund', 'Total'])]

    non_nu_values['rank'] = non_nu_values['Total'].rank(ascending=False)
    non_nu_values.sort_values(by='rank', inplace=True)

    sorted_group = pd.concat([non_nu_values, nu_values])
    return sorted_group.drop(columns='rank')

# Apply the sorting function within each group
df_q_sorted = df_q.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)


#### T_Q tables

In [127]:
# PROCESAREA INTREBARILOR IN FORMAT T_Q

# Assuming df_t is your DataFrame
# Drop rows where 'question' is NaN
df_t = df_t.dropna(subset=['Răspunsuri'])

# Identify the rows where a new question starts, making the search case-insensitive and stripping any whitespace
question_indices_t = df_t[df_t['Răspunsuri'].str.strip().str.startswith('T_Q')].index



# Iterate through each question section_t
for i in range(len(question_indices_t) - 1):
    start_idx_t = question_indices_t[i]
    end_idx_t = question_indices_t[i + 1]

    # Extract the relevant section_t of the DataFrame
    section_t = df_t.loc[start_idx_t+1:end_idx_t]

    # Convert columns to numeric (excluding the first 3 columns)
    section_t.iloc[:, 3:] = section_t.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')

    # Calculate the sum of each column in the section_t (excluding the first 3 columns)
    col_sums = section_t.iloc[:, 3:].sum(axis=0)

    # Avoid division by zero
    col_sums_nonzero = col_sums.replace(0, 1)  # Replace zeros with 1 to avoid division by zero
    section_t.iloc[:, 3:] = section_t.iloc[:, 3:].div(col_sums_nonzero, axis=1) * 100

    # Replace NaN values with 0 after division
    section_t.iloc[:, 3:].fillna(0, inplace=True)

    # Update the original DataFrame with the calculated values
    df_t.loc[start_idx_t+1:end_idx_t] = section_t

# Corrected process for the last section
start_idx_t = question_indices_t[-1]  # Use the correct list for indices
section_t = df_t.loc[start_idx_t+1:]

# Repeat the normalization steps for the last section
section_t.iloc[:, 3:] = section_t.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')
col_sums = section_t.iloc[:, 3:].sum(axis=0)
col_sums_nonzero = col_sums.replace(0, np.nan)  # Consider using NaN instead of replacing with 1
section_t.iloc[:, 3:] = section_t.iloc[:, 3:].div(col_sums_nonzero, axis=1).multiply(100)
section_t.iloc[:, 3:].fillna(0, inplace=True)
df_t.loc[start_idx_t+1:] = section_t

# Assuming df_t is your DataFrame
# Find the index of the "Relative Frequency" column
relative_frequency_index_t = df_t.columns.get_loc("Total")

# Select columns after the "Relative Frequency" column
columns_to_round_t = df_t.columns[relative_frequency_index_t + 1:]

# Iterate through the columns, convert to numeric, and round
for column_t in columns_to_round_t:
    # Try to convert the column to numeric
    df_t[column_t] = pd.to_numeric(df_t[column_t], errors='coerce')

    # Round the numeric values in the column
    df_t[column_t] = df_t[column_t].round(1)

# Reset the index of the DataFrame
df_t.reset_index(drop=True, inplace=True)

# Create a new column named 'q' as the first column
df_t.insert(0, 'q', '')

# Iterate through the DataFrame to find rows that start with 'T_Q'
for index_t, row_t in df_t.iterrows():
    if row_t['Răspunsuri'].startswith('T_Q'):
        # Copy the value in the corresponding row to the 'q' column
        df_t.at[index_t, 'q'] = row_t['Răspunsuri']

# Forward fill values in the 'q' column
df_t['q'] = df_t['q'].replace('', method='ffill')

# Drop rows where 'question' column starts with 'T_Q'
df_t = df_t[~df_t['Răspunsuri'].str.startswith('T_Q')]

# Reset the index after dropping rows
df_t = df_t.reset_index(drop=True)

# Assuming df_t is your DataFrame
# Create a new DataFrame to store the results
df_t_result = pd.DataFrame(columns=df_t.columns)

# Iterate through each unique question in the 'q' column
for question_t in df_t['q'].unique():
    # Filter the DataFrame for the current question
    df_subset_t = df_t[df_t['q'] == question_t]

    # Calculate the total for each column and insert it as a new row
    total_row_t = df_subset_t.sum(axis=0)
    total_row_t['Răspunsuri'] = "Total"

    # Append the total row to the result DataFrame
    df_t_result = pd.concat([df_t_result, df_subset_t, pd.DataFrame([total_row_t])], ignore_index=True)

# Reset the index of the result DataFrame
df_t_result.reset_index(drop=True, inplace=True)

# Convert 'Total' and 'Relative Frequency' columns to numeric
df_t['N'] = pd.to_numeric(df_t['N'], errors='coerce')
df_t['Total'] = pd.to_numeric(df_t['Total'], errors='coerce')

# Call the function to adjust all columns to the right of 'Relative Frequency'
df_t = adjust_total_column(df_t)

# SORTARE CU NU STIU/NU RASPUND
# Function to sort each group by 'Relative Frequency' in descending order placing 'Nu ştiu' and 'Nu răspund' at the bottom

# Apply the sorting function within each group for df_t
df_t_sorted = df_t.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)

#### Q_option tables

In [128]:
# # PROCESAREA INTREBARILOR CU ALEGERI MULTIPLE Q_O_

def adjust_total_column_multiple(df, target_sum=100):
    # Create a copy of the DataFrame to avoid modifying the original
    adjusted_df = df.copy()

    # Fill NaN values with 0 for the 'Total' column
    adjusted_df['Total'].fillna(0, inplace=True)

    # Iterate through the unique questions
    for q in adjusted_df['q'].unique():
        # Filter rows for the current question
        q_rows = adjusted_df[adjusted_df['q'] == q]

        # Calculate the sum for the 'Total' column for the current question
        current_sum = q_rows['Total'].sum()

        # Check if the sum is non-zero before making adjustments
        if current_sum != 0:
            # Calculate the difference from the target sum
            difference = target_sum - current_sum

            # Adjust the maximum value based on whether the sum was less or more than the target sum
            adjusted_df.loc[q_rows['Total'].idxmax(), 'Total'] += difference

    return adjusted_df

# Use apply to call the function only for the 'Relative Frequency' column
results_multiple = adjust_total_column_multiple(results_multiple)


# Apply the sorting function within each group
rm_sorted = results_multiple.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)



## Preliminary Tables Formating

#### Dissagregation columns order

In [129]:
# Define your specific order for age groups
specific_order_age = ['18-29', '30-37', '38-47', '48-57', '58-66', '67+']

# Define your specific order for regions
specific_order_region = ['Nord', 'Centru', 'Mun. Chișinău', 'Sud', 'Găgăuzia']

specific_order_m = ['Oraș','Sat','Necunoscut']

specific_order_cat_sua = ['Russian speakers 15-29 years.','Russian speakers 30-39 years.','Romanian speakers 15-29 years.']

# Define unique_values_list based on unique values in disaggregation columns
unique_values_list = []
for col in disaggregation_columns:
    for item in df_dummies[col].unique():
        if item not in unique_values_list:
            unique_values_list.append(item)

# Filter specific_order_age to include only those values that are present in unique_values_list
filtered_specific_order_age = [item for item in specific_order_age if item in unique_values_list]

# Filter specific_order_region to include only those values that are present in unique_values_list
filtered_specific_order_region = [item for item in specific_order_region if item in unique_values_list]

# Filter specific_order_m to include only those values that are present in unique_values_list
filtered_specific_order_m = [item for item in specific_order_m if item in unique_values_list]

# Filter specific_order_cat_sua to include only those values that are present in unique_values_list
filtered_specific_order_cat_sua = [item for item in specific_order_cat_sua if item in unique_values_list]


# Combine the filtered specific orders
combined_filtered_specific_order = filtered_specific_order_age + filtered_specific_order_region + filtered_specific_order_m + specific_order_cat_sua

# Now, place the values in combined_filtered_specific_order at the beginning of unique_values_list in the specified order
# And include other items not in combined_filtered_specific_order
unique_values_list = combined_filtered_specific_order + [item for item in unique_values_list if item not in combined_filtered_specific_order]

#### Q_digit tables

In [130]:


# FORMATAREA INTREBARILOR Q_DIGIT

# Identify numeric columns (excluding non-numeric columns)
numeric_columns = df_q_sorted.select_dtypes(include=['number']).columns

# Convert 'N' and 'Total' columns to numeric
df_q_sorted['N'] = pd.to_numeric(df_q_sorted['N'], errors='coerce')
df_q_sorted['Total'] = pd.to_numeric(df_q_sorted['Total'], errors='coerce')

# Create a 'Total' row for each unique question
totals = df_q_sorted.groupby('q')[numeric_columns].sum().reset_index()
totals['Răspunsuri'] = 'Total'


# Identify the unique questions
unique_questions = df_q_sorted['q'].unique()

# Initialize an empty DataFrame to store the result
result_df_q = pd.DataFrame(columns=df_q.columns)

# Iterate through unique questions and insert the 'N' row after each group
for q in unique_questions:
    question_rows = df_q[df_q['q'] == q]
    result_df_q = pd.concat([result_df_q, question_rows, totals[totals['q'] == q]], ignore_index=True)

def add_empty_rows_and_copy_labels(df):
    # Create a mask to identify the unique question labels
    mask = df['q'] != df['q'].shift(1)

    # Define a function to add three empty rows for each group
    def add_empty_rows_and_copy_labels_func(group):
        empty_rows = pd.DataFrame(columns=df.columns, data=[[None] * len(df.columns)] * 3)

        # Insert the label of the unique question group in the 'question' column of the second empty row
        empty_rows.at[1, 'Răspunsuri'] = group['q'].iloc[1]

        # Copy the column names and insert them above each group
        empty_rows.iloc[[0, 2], :] = df.columns

        return pd.concat([empty_rows, group.reset_index(drop=True)], ignore_index=True)

    # Apply the function to each group of unique questions
    result_df = df.groupby('q').apply(add_empty_rows_and_copy_labels_func).reset_index(drop=True)

    return result_df

result_df_q= result_df_q.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)

# Example usage:
result_df_q = add_empty_rows_and_copy_labels(result_df_q)



# Specify the starting index for the replacement
start_index = 1

# Iterate through the 'q' column and replace entire rows for every second 'q' value starting from the specified index
replace_count = 0
for index, value in result_df_q['q'].items():
    if index >= start_index and value == 'q':
        if replace_count % 2 == 1:
            result_df_q.iloc[index, :] = [None] * len(result_df_q.columns)
        replace_count += 1

# Cut the 'Total' column
relative_frequency_col = result_df_q.pop('Total')

# Insert the 'Total' column just after the 'N' column
result_df_q.insert(result_df_q.columns.get_loc('N') + 1, 'Total', relative_frequency_col)


# Create an empty DataFrame to store the result
result_df_q_m = pd.DataFrame(columns=result_df_q.columns)


# Iterate through the original DataFrame
for index, row in result_df_q.iterrows():
    # If the current row's question starts with 'Q_', insert an empty row above
    if row['Răspunsuri'] is not None and row['Răspunsuri'].startswith('Q_'):
        result_df_q_m = result_df_q_m.append(pd.Series(), ignore_index=True)

    # Append the current row to the result DataFrame
    result_df_q_m = result_df_q_m.append(row)

    # If the current row's question starts with 'Q_', insert an empty row below
    if row['Răspunsuri'] is not None and row['Răspunsuri'].startswith('Q_'):
        result_df_q_m = result_df_q_m.append(pd.Series(), ignore_index=True)

# Reset the index of the result DataFrame
result_df_q_m.reset_index(drop=True, inplace=True)

# Assuming 'Total' is the column after which you want to reorder
total_column_index = result_df_q_m.columns.get_loc('Total')

# Separate the DataFrame into three parts: before 'Total', 'Total', and after 'Total'
columns_before_total = result_df_q_m.columns[:total_column_index]
total_column = result_df_q_m.columns[total_column_index:total_column_index + 1]
columns_after_total = result_df_q_m.columns[total_column_index + 1:]

# Reorder the columns after 'Total' based on unique_values_list
# Only include columns that are present in the DataFrame
reordered_columns_after_total = [col for col in unique_values_list if col in columns_after_total]

# Combine the columns back into the new order
new_column_order = list(columns_before_total) + list(total_column) + reordered_columns_after_total

# Reorder the DataFrame columns
result_df_q_m = result_df_q_m[new_column_order]



# Create a new DataFrame with the same columns and an empty first row
empty_row_df = pd.DataFrame(columns=result_df_q_m.columns, index=[0])

# Update the first row of result_df_q_m with the empty values
result_df_q_m.iloc[0] = empty_row_df.iloc[0]

# # Prompt the user to input the value they want to insert
# value_to_insert = input("Introduceti denumirea grupei de dezagregare: ")

value_to_insert = 'Grupe de dezagregare (%)'

# Assuming 'Total' is the column after which you want to insert the value
column_name = 'Total'
column_index = result_df_q_m.columns.get_loc(column_name)

# Calculate the index for the column after 'Total'
insert_column_index = column_index + 1

# Check if the insert_column_index is within the DataFrame's column range
if insert_column_index < len(result_df_q_m.columns):
    empty_row_count = 0

    # Iterate through the DataFrame
    for index, row in result_df_q_m.iterrows():
        if pd.isnull(row.iloc[insert_column_index]):
            empty_row_count += 1
            if empty_row_count == 4:
                # Insert the value into the DataFrame at the specified column index
                result_df_q_m.iat[index, insert_column_index] = value_to_insert
                # break  # Stop after the first insertion
        else:
            # Reset the counter if a non-empty value is encountered
            empty_row_count = 0

# If the insert_column_index is not valid, this part of the script will do nothing



result_df_q_m.drop('q', axis=1, inplace=True)




# ATRIBUIREA INTREBARILOR

# Create a dictionary mapping Variable Name to Original Text
mapping_dict = questions_filtered_df.set_index('q_index')['Text'].to_dict()

# Define a function to replace values in the 'question' column
def replace_question(value):
    if isinstance(value, str):
        match = re.match(r'Q_(\d+)', value)
        if match:
            question_number = match.group(1)
            return f'Întrebarea {question_number} : {mapping_dict.get(value, "")}'
    return value



# Apply the custom function to the 'question' column
result_df_q_m['Răspunsuri'] = result_df_q_m['Răspunsuri'].apply(replace_question)

#### T_Q tabels

In [131]:
# FORMATAREA INTREBARILOR T_Q


# Identify numeric columns (excluding non-numeric columns)
numeric_columns_t = df_t_sorted.select_dtypes(include=['number']).columns

# Convert 'N' and 'Total' columns to numeric
df_t_sorted['N'] = pd.to_numeric(df_t_sorted['N'], errors='coerce')
df_t_sorted['Total'] = pd.to_numeric(df_t_sorted['Total'], errors='coerce')

# Create a 'Total' row for each unique question
totals_t = df_t_sorted.groupby('q')[numeric_columns_t].sum().reset_index()
totals_t['Răspunsuri'] = 'Total'

# Identify the unique questions
unique_questions_t = df_t_sorted['q'].unique()


# Initialize an empty DataFrame to store the result
result_df_t = pd.DataFrame(columns=df_t.columns)

# Iterate through unique questions and insert the 'Total' row after each group
for q_t in unique_questions_t:
    question_rows_t = df_t[df_t['q'] == q_t]
    result_df_t = pd.concat([result_df_t, question_rows_t, totals_t[totals_t['q'] == q_t]], ignore_index=True)

result_df_t = result_df_t.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)

# Example usage:
result_df_t = add_empty_rows_and_copy_labels(result_df_t)

# Specify the starting index for the replacement
start_index_t = 1

# Iterate through the 'q' column and replace entire rows for every second 'q' value starting from the specified index
replace_count_t = 0
for index_t, value_t in result_df_t['q'].items():
    if index_t >= start_index_t and value_t == 'q':
        if replace_count_t % 2 == 1:
            result_df_t.iloc[index_t, :] = [None] * len(result_df_t.columns)
        replace_count_t += 1

# Cut the 'Total' column
relative_frequency_col_t = result_df_t.pop('Total')

# Insert the 'Total' column just after the 'N' column
result_df_t.insert(result_df_t.columns.get_loc('N') + 1, 'Total', relative_frequency_col_t)

# Create an empty DataFrame to store the result
result_df_t_m = pd.DataFrame(columns=result_df_t.columns)

# Iterate through the original DataFrame
for index_t, row_t in result_df_t.iterrows():
    # If the current row's question starts with 'T_Q', insert an empty row above
    if row_t['Răspunsuri'] is not None and row_t['Răspunsuri'].startswith('T_Q'):
        result_df_t_m = result_df_t_m.append(pd.Series(), ignore_index=True)

    # Append the current row to the result DataFrame
    result_df_t_m = result_df_t_m.append(row_t)

    # If the current row's question starts with 'T_Q', insert an empty row below
    if row_t['Răspunsuri'] is not None and row_t['Răspunsuri'].startswith('T_Q'):
        result_df_t_m = result_df_t_m.append(pd.Series(), ignore_index=True)

# Reset the index of the result DataFrame
result_df_t_m.reset_index(drop=True, inplace=True)


# Assuming 'Total' is the column after which you want to reorder
total_column_index = result_df_t_m.columns.get_loc('Total')

# Separate the DataFrame into three parts: before 'Total', 'Total', and after 'Total'
columns_before_total = result_df_t_m.columns[:total_column_index]
total_column = result_df_t_m.columns[total_column_index:total_column_index + 1]
columns_after_total = result_df_t_m.columns[total_column_index + 1:]

# Reorder the columns after 'Total' based on unique_values_list
# Only include columns that are present in the DataFrame
reordered_columns_after_total = [col for col in unique_values_list if col in columns_after_total]

# Combine the columns back into the new order
new_column_order = list(columns_before_total) + list(total_column) + reordered_columns_after_total

# Reorder the DataFrame columns
result_df_t_m = result_df_t_m[new_column_order]


# Create a new DataFrame with the same columns and an empty first row
empty_row_df_t = pd.DataFrame(columns=result_df_t_m.columns, index=[0])

# Update the first row of result_df_t_m with the empty values
result_df_t_m.iloc[0] = empty_row_df_t.iloc[0]

value_to_insert_t = value_to_insert

# Assuming 'Total' is the column after which you want to insert the value
column_name = 'Total'
column_index = result_df_t_m.columns.get_loc(column_name)

# Calculate the index for the column after 'Total'
insert_column_index = column_index + 1

# Check if the insert_column_index is within the DataFrame's column range
if insert_column_index < len(result_df_t_m.columns):
    empty_row_count = 0

    # Iterate through the DataFrame
    for index, row in result_df_t_m.iterrows():
        if pd.isnull(row.iloc[insert_column_index]):
            empty_row_count += 1
            if empty_row_count == 4:
                # Insert the value into the DataFrame at the specified column index
                result_df_t_m.iat[index, insert_column_index] = value_to_insert
                #break  # Stop after the first insertion
        else:
            # Reset the counter if a non-empty value is encountered
            empty_row_count = 0

# If the insert_column_index is not valid, this part of the script will do nothing


result_df_t_m.drop('q', axis=1, inplace=True)

# Loop through the DataFrame
for i in range(1, len(result_df_t_m)):
    value = result_df_t_m.loc[i, 'Răspunsuri']
    
    # Check if the value is a string and starts with 'T_Q'
    if isinstance(value, str) and value.startswith('T_Q'):
        # Extract the required part of the string
        extracted_value = '_'.join(value.split('_')[:3])
        
        # Assign the extracted value to the previous row
        result_df_t_m.loc[i - 1, 'Răspunsuri'] = extracted_value

# ATRIBUIREA INTREBARILOR

# Create a dictionary mapping Variable Name to Original Text
mapping_dict = questions_filtered_df.set_index('q_index')['Text'].to_dict()

def replace_question(value, mapping_dict):
    if isinstance(value, str):
        # Pattern for 'T_Q_123_456'
        match = re.match(r'T_Q_(\d+)_(\d+)', value)
        if match:
            question_number = match.group(1)
            replacement = f'opțiunea : {mapping_dict.get(value, "Unknown")}'
            return replacement

        # Pattern for 'T_Q_123'
        match = re.match(r'T_Q_(\d+)', value)
        if match:
            question_number = match.group(1)
            replacement = f'Întrebarea {question_number} : {mapping_dict.get(value, "Unknown")}'
            return replacement

    return value

# Apply the custom function to the 'question' column
result_df_t_m['Răspunsuri'] = result_df_t_m['Răspunsuri'].apply(lambda x: replace_question(x, mapping_dict))


#### Q_option tables

In [132]:
# FORMATAREA INTREBARILOR Q_Option


# Identify numeric columns (excluding non-numeric columns)
numeric_columns_rm = rm_sorted.select_dtypes(include=['number']).columns

# Convert 'N' and 'Total' columns to numeric
rm_sorted['N'] = pd.to_numeric(rm_sorted['N'], errors='coerce')
rm_sorted['Total'] = pd.to_numeric(rm_sorted['Total'], errors='coerce')


# Create a 'Total' row for each unique question
totals_rm = rm_sorted.groupby('q')[numeric_columns_rm].sum().reset_index()
totals_rm['Răspunsuri'] = 'Total'

# Identify the unique questions
unique_questions_rm = rm_sorted['q'].unique()


# Apply the sorting function within each group
rm_sorted = results_multiple.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)


# Initialize an empty DataFrame to store the result
result_rm = pd.DataFrame(columns=df_t.columns)

# Iterate through unique questions and insert the 'Total' row after each group
for q_rm in unique_questions_rm:
    question_rows_rm = rm_sorted[rm_sorted['q'] == q_rm]
    result_rm = pd.concat([result_rm, question_rows_rm, totals_rm[totals_rm['q'] == q_rm]], ignore_index=True)

result_rm = result_rm.groupby('q', group_keys=False).apply(sort_within_group).reset_index(drop=True)

# Example usage:
result_rm = add_empty_rows_and_copy_labels(result_rm)

start_index_rm = 1

# Iterate through the 'q' column and replace entire rows for every second 'q' value starting from the specified index
replace_count_rm = 0
for index_rm, value_rm in result_rm['q'].items():
    if index_rm >= start_index_rm and value_rm == 'q':
        if replace_count_rm % 2 == 1:
            result_rm.iloc[index_rm, :] = [None] * len(result_rm.columns)
        replace_count_rm += 1


# Create an empty DataFrame to store the result
result_rm_m = pd.DataFrame(columns=result_rm.columns)

# Iterate through the original DataFrame
for index_rm, row_rm in result_rm.iterrows():
    # If the current row's question starts with 'Q', insert an empty row above
    if row_rm['Răspunsuri'] is not None and row_rm['Răspunsuri'].startswith('Q'):
        result_rm_m = result_rm_m.append(pd.Series(), ignore_index=True)

    # Append the current row to the result DataFrame
    result_rm_m = result_rm_m.append(row_rm)

    # If the current row's question starts with 'T_Q', insert an empty row below
    if row_rm['Răspunsuri'] is not None and row_rm['Răspunsuri'].startswith('Q'):
        result_rm_m = result_rm_m.append(pd.Series(), ignore_index=True)

# Reset the index of the result DataFrame
result_rm_m.reset_index(drop=True, inplace=True)

# Assuming 'Total' is the column after which you want to reorder
total_column_index = result_rm_m.columns.get_loc('Total')

# Separate the DataFrame into three parts: before 'Total', 'Total', and after 'Total'
columns_before_total = result_rm_m.columns[:total_column_index]
total_column = result_rm_m.columns[total_column_index:total_column_index + 1]
columns_after_total = result_rm_m.columns[total_column_index + 1:]

# Reorder the columns after 'Total' based on unique_values_list
# Only include columns that are present in the DataFrame
reordered_columns_after_total = [col for col in unique_values_list if col in columns_after_total]

# Combine the columns back into the new order
new_column_order = list(columns_before_total) + list(total_column) + reordered_columns_after_total

# Reorder the DataFrame columns
result_rm_m = result_rm_m[new_column_order]



# Create a new DataFrame with the same columns and an empty first row
empty_row_rm = pd.DataFrame(columns=result_rm_m.columns, index=[0])

# Update the first row of result_df_t_m with the empty values
result_rm_m.iloc[0] = empty_row_rm.iloc[0]


value_to_insert_rm = value_to_insert

# Assuming 'Total' is the column after which you want to insert the value
column_name = 'Total'
column_index = result_rm_m.columns.get_loc(column_name)

# Calculate the index for the column after 'Total'
insert_column_index = column_index + 1

# Check if the insert_column_index is within the DataFrame's column range
if insert_column_index < len(result_rm_m.columns):
    empty_row_count = 0

    # Iterate through the DataFrame
    for index, row in result_rm_m.iterrows():
        if pd.isnull(row.iloc[insert_column_index]):
            empty_row_count += 1
            if empty_row_count == 4:
                # Insert the value into the DataFrame at the specified column index
                result_rm_m.iat[index, insert_column_index] = value_to_insert
                #break  # Stop after the first insertion
        else:
            # Reset the counter if a non-empty value is encountered
            empty_row_count = 0

# If the insert_column_index is not valid, this part of the script will do nothing


result_rm_m.drop('q', axis=1, inplace=True)

# Assuming result_rm_m is your DataFrame and 'Răspunsuri' is the column you want to modify
result_rm_m['Răspunsuri'] = result_rm_m['Răspunsuri'].apply(
    lambda x: x[0] + '_' + x[1:] if isinstance(x, str) and x.startswith('Q') else x
)

# ATRIBUIREA INTREBARILOR

mapping_dict = questions_filtered_df.set_index('q_index')['Text'].to_dict()


def replace_question(value):
    if isinstance(value, str):
        match = re.match(r'Q_(\d+)', value)
        if match:
            question_number = match.group(1)
            replacement = f'Întrebarea {question_number} : {mapping_dict.get(value, "")}'
            return replacement
    return value

result_rm_m['Răspunsuri'] = result_rm_m['Răspunsuri'].apply(replace_question)


## Sorting

##### Q digit

In [133]:


# Function to identify sections
def identify_sections(df, column_name):
    sections = []
    start = None
    for i, row in df.iterrows():
        if pd.isna(row[column_name]) and start is not None:
            sections.append((start, i))
            start = None
        elif not pd.isna(row[column_name]) and start is None:
            start = i
    if start is not None:
        sections.append((start, len(df)))
    return sections

def sort_section(df, start, end):
    section = df.iloc[start:end].copy()

    section['N_sort'] = pd.to_numeric(section['N'], errors='coerce')

    def matches_pattern(x, patterns):
        return any(re.search(pattern, str(x), re.IGNORECASE) for pattern in patterns)

    top_rows = section['Răspunsuri'].apply(lambda x: matches_pattern(x, ['^Răspunsuri$']))

    # Improved regex for 'Nu știu' to capture common variations
    # The pattern covers: optional whitespace, optional diacritics, and slight misspellings
    nu_stiu_pattern = r'Nu\s*[șş]{1}tiu.*'
    nu_raspund_pattern = r'Nu (vreau să )?răspund'
    bottom_rows = section['Răspunsuri'].apply(lambda x: matches_pattern(x, ['Total', r'Alt\w*', nu_stiu_pattern, nu_raspund_pattern]))


    other_rows = ~top_rows & ~bottom_rows

    sorted_section = section[other_rows].sort_values(by='N_sort', ascending=False, na_position='last')

    final_section = pd.concat([section[top_rows], sorted_section, section[bottom_rows]])

    final_section = final_section.drop(columns=['N_sort'])

    if len(final_section) != len(section):
        raise ValueError("Mismatch in section lengths after sorting.")

    df.iloc[start:end] = final_section.values




# Identify sections in the DataFrame
sections = identify_sections(result_df_q_m, 'Răspunsuri')

# Sort each section
for start, end in sections:
    sort_section(result_df_q_m, start, end)

##### Q multiple

In [134]:
# Function to identify sections
def identify_sections(df, column_name):
    sections = []
    start = None
    for i, row in df.iterrows():
        if pd.isna(row[column_name]) and start is not None:
            sections.append((start, i))
            start = None
        elif not pd.isna(row[column_name]) and start is None:
            start = i
    if start is not None:
        sections.append((start, len(df)))
    return sections

def sort_section(df, start, end):
    section = df.iloc[start:end].copy()

    section['N_sort'] = pd.to_numeric(section['N'], errors='coerce')

    def matches_pattern(x, patterns):
        return any(re.search(pattern, str(x), re.IGNORECASE) for pattern in patterns)

    top_rows = section['Răspunsuri'].apply(lambda x: matches_pattern(x, ['^Răspunsuri$']))

    # Improved regex for 'Nu știu' to capture common variations
    nu_stiu_pattern = r'Nu\s*[șş]{1}tiu.*'
    nu_raspund_pattern = r'Nu (vreau să )?răspund'
    bottom_rows = section['Răspunsuri'].apply(lambda x: matches_pattern(x, ['Total', r'Alt\w*', nu_stiu_pattern, nu_raspund_pattern]))


    other_rows = ~top_rows & ~bottom_rows

    sorted_section = section[other_rows].sort_values(by='N_sort', ascending=False, na_position='last')

    final_section = pd.concat([section[top_rows], sorted_section, section[bottom_rows]])

    final_section = final_section.drop(columns=['N_sort'])

    if len(final_section) != len(section):
        raise ValueError("Mismatch in section lengths after sorting.")

    df.iloc[start:end] = final_section.values




# Identify sections in the DataFrame
sections = identify_sections(result_rm_m, 'Răspunsuri')

# Apply the sorting to result_rm_m
for start, end in sections:
    sort_section(result_rm_m, start, end)


##### T_Q

In [135]:
# Function to identify sections
def identify_sections(df, column_name):
    sections = []
    start = None
    for i, row in df.iterrows():
        if pd.isna(row[column_name]) and start is not None:
            sections.append((start, i))
            start = None
        elif not pd.isna(row[column_name]) and start is None:
            start = i
    if start is not None:
        sections.append((start, len(df)))
    return sections

def sort_section_t(df, start, end):
    section = df.iloc[start:end].copy()

    # Helper function to extract the numeric part from a string for sorting
    def extract_numeric_part(s):
        match = re.search(r'(\d+)', s)
        return int(match.group(1)) if match else float('inf')  # Using infinity for strings without a numeric part

    # Identify the rows
    top_rows = section['Răspunsuri'] == 'Răspunsuri'
    numeric_rows = section['Răspunsuri'].str.contains(r'^\d+', regex=True)
    total_rows = section['Răspunsuri'] == 'Total'
    nu_stiu_pattern = r'Nu\s*[șş]{1}tiu.*'
    nu_stiu_rows = section['Răspunsuri'].str.contains(nu_stiu_pattern, regex=True)
    # 'Nu vreau sa răspund' and 'Nu răspund' should be considered the same
    nu_raspund_pattern = r'Nu (vreau să )?răspund'
    nu_raspund_rows = section['Răspunsuri'].str.contains(nu_raspund_pattern, regex=True)


    # Sort the numeric rows based on the numeric part
    section['Numeric_Part'] = section['Răspunsuri'].apply(extract_numeric_part)
    sorted_numeric_rows = section[numeric_rows].sort_values(by='Numeric_Part')

    # Combine the rows in the desired order
    final_section = pd.concat([
        section[top_rows],
        sorted_numeric_rows,
        section[~top_rows & ~numeric_rows & ~total_rows & ~nu_stiu_rows & ~nu_raspund_rows],
        section[nu_raspund_rows],
        section[nu_stiu_rows],
        section[total_rows]
    ])

    final_section = final_section.drop(columns=['Numeric_Part'], errors='ignore')

    if len(final_section) != len(section):
        raise ValueError("Mismatch in section lengths after sorting.")

    df.iloc[start:end] = final_section.values





# Identify sections in the DataFrame
sections = identify_sections(result_df_t_m, 'Răspunsuri')

# Apply the sorting to result_rm_m
for start, end in sections:
    sort_section_t(result_df_t_m, start, end)

## Excel formating and export

### Q_digit tables

In [136]:
# Create a new Workbook
wb = Workbook()
ws = wb.active
# Give a name to the sheet
ws.title = "Intrebari_Q"

# Keep track of the last formatted cell
last_formatted_cell = None

# Iterate through the 'question' column
for index, value in enumerate(result_df_q_m['Răspunsuri']):
    cell = ws.cell(row=index + 2, column=1, value=value)

    # Check if value is None
    if value is not None:
        # Convert 'value' to a string if it's a float
        if isinstance(value, float):
            value = str(value)

        # Set the font and size
        cell.font = Font(name='Arial', size=10 if value.startswith('Întrebare') else 9, bold=True)

        # Set the alignment
        if value == 'Răspunsuri':
            cell.alignment = Alignment(horizontal='center', vertical='center')
        elif value == 'Total':
            cell.alignment = Alignment(horizontal='right', vertical='center')
        else:
            cell.alignment = Alignment(horizontal='left', vertical='center')

        # Keep track of the last formatted cell
        last_formatted_cell = cell

for index, value in enumerate(result_df_q_m['Răspunsuri']):
    if value == 'Răspunsuri' and index > 0:  # Check if it's not the first row
        ws.merge_cells(start_row=index + 1, start_column=1, end_row=index + 2, end_column=1)
        ws.cell(row=index + 1, column=1).value = 'Răspunsuri'  # Remove this line
        ws.cell(row=index +1 , column=1).alignment = Alignment(horizontal='center', vertical='center')


# Iterate through the 'N' column
for index, value in enumerate(result_df_q_m['N']):
    cell = ws.cell(row=index + 2, column=2, value=value)

    # Set the font and alignment based on value
    if value == 'N':
        cell.font = Font(name='Arial', size=9, bold=True)
        cell.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell.font = Font(name='Arial', size=9, bold=False)
        cell.alignment = Alignment(horizontal='right', vertical='center')

# Perform the merge after formatting the entire 'N' column
for index, value in enumerate(result_df_q_m['N']):
    if value == 'N' and index > 0:  # Check if it's not the first row
        ws.merge_cells(start_row=index + 1, start_column=2, end_row=index + 2, end_column=2)
        ws.cell(row=index + 1, column=2).value = 'N'  # Set the merged cell value to 'N'
        ws.cell(row=index + 1, column=2).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style = Font(name='Arial', size=9, bold=True)
        ws.cell(row=index + 1, column=2).font = font_style


# Iterate through the 'Total' column
for index, value in enumerate(result_df_q_m['Total']):
    cell = ws.cell(row=index + 2, column=3, value=value)

    # Set the font and alignment based on value
    if value == 'Total':
        cell.font = Font(name='Arial', size=9, bold=True)
        cell.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell.font = Font(name='Arial', size=9, bold=False)
        cell.alignment = Alignment(horizontal='right', vertical='center')
        # Format the value to '100.0' if it's not the header row
        cell.number_format = '0.0'

# Perform the merge after formatting the entire 'Total' column
for index, value in enumerate(result_df_q_m['Total']):
    if value == 'Total' and index > 0:  # Check if it's not the first row
        ws.merge_cells(start_row=index + 1, start_column=3, end_row=index + 2, end_column=3)
        ws.cell(row=index + 1, column=3).value = 'Total (%)'  # Set the merged cell value to 'Total'
        ws.cell(row=index + 1, column=3).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style = Font(name='Arial', size=9, bold=True)
        ws.cell(row=index + 1, column=3).font = font_style



# Iterate through the unique values list only if it's not empty
if unique_values_list:
    for col_index, col_name in enumerate(unique_values_list):
        # Check if col_name exists in the DataFrame columns
        if col_name in result_df_q_m.columns:
            # Iterate through the column
            for index, value in enumerate(result_df_q_m[col_name]):
                cell = ws.cell(row=index + 2, column=col_index + 4, value=value)

                # Set the font and alignment based on value
                cell.font = Font(name='Arial', size=9, bold=True if str(value) in unique_values_list else False)
                cell.alignment = Alignment(horizontal='center' if str(value) in unique_values_list else 'right', vertical='center')
                if str(value) not in unique_values_list:
                    cell.number_format = '0.0'
    # Find the column name corresponding to the first value in unique_values_list
    target_column_name = unique_values_list[0]
    # Check if unique_values_list is not empty and value_to_insert is in the target column
    if unique_values_list and value_to_insert in result_df_q_m.get(target_column_name, []):
        for index_to_merge in [i for i, value in enumerate(result_df_q_m[target_column_name]) if value == value_to_insert]:
            for col_index, col_name in enumerate(unique_values_list):
                if col_name == target_column_name:
                    ws.merge_cells(start_row=index_to_merge + 2, start_column=col_index + 4, end_row=index_to_merge + 2, end_column=col_index + 4 + len(unique_values_list) - 1)
                    cell_to_insert = ws.cell(row=index_to_merge + 2, column=col_index + 4)
                    cell_to_insert.value = value_to_insert
                    cell_to_insert.font = Font(name='Arial', size=9, bold=True)
                    cell_to_insert.alignment = Alignment(horizontal='center', vertical='center')



### T_Q tables

In [137]:
# Create a new Workbook for the second sheet
ws_t = wb.create_sheet(title='Intrebari_T_Q')

# Keep track of the last formatted cell for result_df_t_m
last_formatted_cell_t = None

# Iterate through the 'Răspunsuri' column for result_df_t_m and format cells
for index_t, value_t in enumerate(result_df_t_m['Răspunsuri']):
    cell_t = ws_t.cell(row=index_t + 2, column=1, value=value_t)

    # Check if value is None
    if value_t is not None:
        # Convert 'value' to a string if it's a float
        if isinstance(value_t, float):
            value_t = str(value_t)

        # Set the font and size
        cell_t.font = Font(name='Arial', size=10 if value_t.startswith('Întrebare') else 9, bold=True)

        # Set the alignment
        if value_t == 'Răspunsuri':
            cell_t.alignment = Alignment(horizontal='center', vertical='center')
        elif value_t == 'Total':
            cell_t.alignment = Alignment(horizontal='right', vertical='center')
        else:
            cell_t.alignment = Alignment(horizontal='left', vertical='center')

        # Keep track of the last formatted cell
        last_formatted_cell_t = cell_t

# Now perform the merging in a separate iteration
for index_t, value_t in enumerate(result_df_t_m['Răspunsuri']):
    if value_t == 'Răspunsuri' and index_t > 0:
        ws_t.merge_cells(start_row=index_t + 1, start_column=1, end_row=index_t + 2, end_column=1)
        ws_t.cell(row=index_t + 1, column=1).value = 'Răspunsuri'  # Remove this line
        ws_t.cell(row=index_t +1 , column=1).alignment = Alignment(horizontal='center', vertical='center')


# Iterate through the 'N' column for result_df_t_m
for index_t, value_t in enumerate(result_df_t_m['N']):
    cell_t = ws_t.cell(row=index_t + 2, column=2, value=value_t)

    # Set the font and alignment based on value
    if value_t == 'N':
        cell_t.font = Font(name='Arial', size=9, bold=True)
        cell_t.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell_t.font = Font(name='Arial', size=9, bold=False)
        cell_t.alignment = Alignment(horizontal='right', vertical='center')

# Perform the merge after formatting the entire 'N' column for result_df_t_m
for index_t, value_t in enumerate(result_df_t_m['N']):
    if value_t == 'N' and index_t > 0:  # Check if it's not the first row
        ws_t.merge_cells(start_row=index_t + 1, start_column=2, end_row=index_t + 2, end_column=2)
        ws_t.cell(row=index_t + 1, column=2).value = 'N'  # Set the merged cell value to 'N'
        ws_t.cell(row=index_t + 1, column=2).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style_t = Font(name='Arial', size=9, bold=True)
        ws_t.cell(row=index_t + 1, column=2).font = font_style_t


# Iterate through the 'Total' column for result_df_t_m
for index_t, value_t in enumerate(result_df_t_m['Total']):
    cell_t = ws_t.cell(row=index_t + 2, column=3, value=value_t)

    # Set the font and alignment based on value
    if value_t == 'Total':
        cell_t.font = Font(name='Arial', size=9, bold=True)
        cell_t.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell_t.font = Font(name='Arial', size=9, bold=False)
        cell_t.alignment = Alignment(horizontal='right', vertical='center')
        # Format the value to '100.0' if it's not the header row
        cell_t.number_format = '0.0'

# Perform the merge after formatting the entire 'Total' column for result_df_t_m
for index_t, value_t in enumerate(result_df_t_m['Total']):
    if value_t == 'Total' and index_t > 0:  # Check if it's not the first row
        ws_t.merge_cells(start_row=index_t + 1, start_column=3, end_row=index_t + 2, end_column=3)
        ws_t.cell(row=index_t + 1, column=3).value = 'Total (%)'  # Set the merged cell value to 'Total'
        ws_t.cell(row=index_t + 1, column=3).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style_t = Font(name='Arial', size=9, bold=True)
        ws_t.cell(row=index_t + 1, column=3).font = font_style_t


# Iterate through the unique values list only if it's not empty
if unique_values_list:
    for col_index_t, col_name_t in enumerate(unique_values_list):
        # Check if col_name exists in the DataFrame columns
        if col_name_t in result_df_t_m.columns:
            # Iterate through the column
            for index_t, value_t in enumerate(result_df_t_m[col_name_t]):
                cell_t = ws_t.cell(row=index_t + 2, column=col_index_t + 4, value=value_t)

                # Set the font and alignment based on value
                cell_t.font = Font(name='Arial', size=9, bold=True if str(value_t) in unique_values_list else False)
                cell_t.alignment = Alignment(horizontal='center' if str(value_t) in unique_values_list else 'right', vertical='center')
                if str(value_t) not in unique_values_list:
                    cell_t.number_format = '0.0'
    # Find the column name corresponding to the first value in unique_values_list
    target_column_name_t = unique_values_list[0]
    # Check if unique_values_list is not empty and if value_to_insert_t is in the target column
    if unique_values_list and value_to_insert_t in result_df_t_m.get(target_column_name_t, []):
        for index_to_merge_t in [i for i, value_t in enumerate(result_df_t_m[target_column_name_t]) if value_t == value_to_insert_t]:
            for col_index_t, col_name_t in enumerate(unique_values_list):
                if col_name_t == target_column_name_t:
                    ws_t.merge_cells(start_row=index_to_merge_t + 2, start_column=col_index_t + 4, end_row=index_to_merge_t + 2, end_column=col_index_t + 4 + len(unique_values_list) - 1)
                    cell_to_insert_t = ws_t.cell(row=index_to_merge_t + 2, column=col_index_t + 4)
                    cell_to_insert_t.value = value_to_insert_t
                    cell_to_insert_t.font = Font(name='Arial', size=9, bold=True)
                    cell_to_insert_t.alignment = Alignment(horizontal='center', vertical='center')


### Q_option tables

In [138]:
# Create a new Workbook for the second sheet
ws_rm = wb.create_sheet(title='Intrebari_Q_multiple')

# Keep track of the last formatted cell for result_rm_m
last_formatted_cell_rm = None

# Iterate through the 'question' column for result_rm_m
for index_rm, value_rm in enumerate(result_rm_m['Răspunsuri']):
    cell_rm = ws_rm.cell(row=index_rm + 2, column=1, value=value_rm)

    # Check if value is None
    if value_rm is not None:
        # Convert 'value' to a string if it's a float
        if isinstance(value_rm, float):
            value_rm = str(value_rm)

        # Set the font and size
        cell_rm.font = Font(name='Arial', size=10 if value_rm.startswith('Întrebare') else 9, bold=True)

        # Set the alignment
        if value_rm == 'Răspunsuri':
            cell_rm.alignment = Alignment(horizontal='center', vertical='center')
        elif value_rm == 'Total':
            cell_rm.alignment = Alignment(horizontal='right', vertical='center')
        else:
            cell_rm.alignment = Alignment(horizontal='left', vertical='center')

        # Keep track of the last formatted cell
        last_formatted_cell_rm = cell_rm

for index_rm, value_rm in enumerate(result_rm_m['Răspunsuri']):
    if value_rm == 'Răspunsuri' and index_rm > 0:  # Check if it's not the first row
        ws_rm.merge_cells(start_row=index_rm + 1, start_column=1, end_row=index_rm + 2, end_column=1)
        ws_rm.cell(row=index_rm + 1, column=1).value = 'Răspunsuri'  # Remove this line
        ws_rm.cell(row=index_rm +1 , column=1).alignment = Alignment(horizontal='center', vertical='center')


# Iterate through the 'N' column for result_rm_m
for index_rm, value_rm in enumerate(result_rm_m['N']):
    cell_rm = ws_rm.cell(row=index_rm + 2, column=2, value=value_rm)

    # Set the font and alignment based on value
    if value_rm == 'N':
        cell_rm.font = Font(name='Arial', size=9, bold=True)
        cell_rm.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell_rm.font = Font(name='Arial', size=9, bold=False)
        cell_rm.alignment = Alignment(horizontal='right', vertical='center')

# Perform the merge after formatting the entire 'N' column for result_rm_m
for index_rm, value_rm in enumerate(result_rm_m['N']):
    if value_rm == 'N' and index_rm > 0:  # Check if it's not the first row
        ws_rm.merge_cells(start_row=index_rm + 1, start_column=2, end_row=index_rm + 2, end_column=2)
        ws_rm.cell(row=index_rm + 1, column=2).value = 'N'  # Set the merged cell value to 'N'
        ws_rm.cell(row=index_rm + 1, column=2).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style_t = Font(name='Arial', size=9, bold=True)
        ws_rm.cell(row=index_rm + 1, column=2).font = font_style_t


# Iterate through the 'Total' column for result_rm_m
for index_rm, value_rm in enumerate(result_rm_m['Total']):
    cell_rm = ws_rm.cell(row=index_rm + 2, column=3, value=value_rm)

    # Set the font and alignment based on value
    if value_rm == 'Total':
        cell_rm.font = Font(name='Arial', size=9, bold=True)
        cell_rm.alignment = Alignment(horizontal='center', vertical='center')
    else:
        cell_rm.font = Font(name='Arial', size=9, bold=False)
        cell_rm.alignment = Alignment(horizontal='right', vertical='center')
        # Format the value to '100.0' if it's not the header row
        cell_rm.number_format = '0.0'

# Perform the merge after formatting the entire 'Total' column for result_rm_m
for index_rm, value_rm in enumerate(result_rm_m['Total']):
    if value_rm == 'Total' and index_rm > 0:  # Check if it's not the first row
        ws_rm.merge_cells(start_row=index_rm + 1, start_column=3, end_row=index_rm + 2, end_column=3)
        ws_rm.cell(row=index_rm + 1, column=3).value = 'Total (%)'  # Set the merged cell value to 'Total'
        ws_rm.cell(row=index_rm + 1, column=3).alignment = Alignment(horizontal='center', vertical='center')
        # Set the font style to Arial, size to 9, and make it bold
        font_style_t = Font(name='Arial', size=9, bold=True)
        ws_rm.cell(row=index_rm + 1, column=3).font = font_style_t


# Iterate through the unique values list only if it's not empty
if unique_values_list:
    for col_index_rm, col_name_rm in enumerate(unique_values_list):
        # Check if col_name exists in the DataFrame columns
        if col_name_rm in result_rm_m.columns:
            # Iterate through the column
            for index_rm, value_rm in enumerate(result_rm_m[col_name_rm]):
                cell_rm = ws_rm.cell(row=index_rm + 2, column=col_index_rm + 4, value=value_rm)

                # Set the font and alignment based on value
                cell_rm.font = Font(name='Arial', size=9, bold=True if str(value_rm) in unique_values_list else False)
                cell_rm.alignment = Alignment(horizontal='center' if str(value_rm) in unique_values_list else 'right', vertical='center')
                if str(value_rm) not in unique_values_list:
                    cell_rm.number_format = '0.0'
    # Find the column name corresponding to the first value in unique_values_list
    target_column_name_rm = unique_values_list[0]
    # Check if unique_values_list is not empty and if value_to_insert_rm is in the target column
    if unique_values_list and value_to_insert_rm in result_rm_m.get(target_column_name_rm, []):
        for index_to_merge_rm in [i for i, value_rm in enumerate(result_rm_m[target_column_name_rm]) if value_rm == value_to_insert_rm]:
            for col_index_rm, col_name_rm in enumerate(unique_values_list):
                if col_name_rm == target_column_name_rm:
                    ws_rm.merge_cells(start_row=index_to_merge_rm + 2, start_column=col_index_rm + 4, end_row=index_to_merge_rm + 2, end_column=col_index_rm + 4 + len(unique_values_list) - 1)
                    cell_to_insert_rm = ws_rm.cell(row=index_to_merge_rm + 2, column=col_index_rm + 4)
                    cell_to_insert_rm.value = value_to_insert_rm
                    cell_to_insert_rm.font = Font(name='Arial', size=9, bold=True)
                    cell_to_insert_rm.alignment = Alignment(horizontal='center', vertical='center')


### Export to excel

In [139]:
# Base file name
base_file_name = str(file_name)

# Modify file name based on disaggregation_columns
if not disaggregation_columns:
    # If disaggregation_columns is empty, add 'Total' to the file name
    base_file_name += '_Total'
else:
    # If disaggregation_columns is not empty, add 'Grupe de dezagragare' to the file name
    base_file_name += '_Grupe_de_dezagragare'

# Further modify file name based on apply_weights
if apply_weights == 'y':
    base_file_name += '_ponderat'

# Add file extension
file_name = base_file_name + '.xlsx'

# Save the workbook with the modified file name
wb.save(file_name)


# Test weights function

In [140]:
# # Function to convert all string characters in a DataFrame to UTF-8
# def convert_to_utf8(df):
#     for col in df.columns:
#         if df[col].dtype == object:  # checks if the column is of type 'object' (typically for strings)
#             df[col] = df[col].apply(lambda x: x.encode('utf-8').decode('utf-8') if isinstance(x, str) else x)
#     return df

# # Convert the DataFrame
# converted_df = convert_to_utf8(df_q)
# converted_df

In [141]:
# converted_df.to_excel('dat.xlsx', index = False)

In [142]:

# def update_weights_for_combined_category(df, combined_categories, target_proportion, current_weights):
#     # Check if current_weights is a Series and has the same length as df
#     if not isinstance(current_weights, pd.Series) or len(current_weights) != len(df):
#         raise ValueError("current_weights must be a pandas Series with the same length as df")

#     # Calculate the count for each category-value pair in combined_categories
#     combined_sample_count = sum(df[df[category] == value].shape[0] for category, value in combined_categories)

#     # Calculate combined sample proportion
#     combined_sample_proportion = combined_sample_count / len(df)
#     print(f"Combined sample proportion: {combined_sample_proportion}")

#     # Avoid division by zero and ensure there's data for combined categories
#     if combined_sample_proportion == 0:
#         raise ValueError("No data for combined categories or zero proportion in data")

#     # # Calculate adjustment factor
#     adjustment_factor = target_proportion / combined_sample_proportion
#     print(f"Adjustment factor: {adjustment_factor}")

#     # Create a copy of the current_weights to modify
#     adjusted_weights = current_weights.copy()

#     # Apply the adjustment factor to the weights of the matching rows
#     for category, value in combined_categories:
#         matching_rows = df[category] == value
#         print(f"Matching rows for category '{category}' and value '{value}':\n{matching_rows}")

#         adjusted_weights[matching_rows] *= adjustment_factor
#         print(f"Adjusted weights after applying factor for '{category}-{value}':\n{adjusted_weights}")

#     return adjusted_weights

# # Define the function to update weights for combined categories
# def update_weights_for_combined_category(df, combined_categories, target_proportion, current_weights):
#     # Combine counts for similar categories
#     combined_sample_count = sum(df[category].value_counts(dropna=False).get(value, 0)
#                                 for category, value in combined_categories)

#     # Calculate combined sample proportion
#     combined_sample_proportion = combined_sample_count / len(df)

#     # Calculate adjustment factor for the combined category
#     adjustment_factor = (target_proportion / combined_sample_proportion 
#                          if combined_sample_proportion > 0 else 1)

#     # Apply the adjustment factor to the weights of the matching rows
#     for category, value in combined_categories:
#         current_weights = current_weights.where(df[category] != value, 
#                                                 current_weights * adjustment_factor)

#     return current_weights


# def test_basic_functionality():
#     # Create a sample DataFrame
#     data = {'category': ['A', 'B', 'A', 'C'],
#             'value': [1, 2, 3, 4]}
#     df = pd.DataFrame(data)
#     current_weights = pd.Series([1, 1, 1, 1], dtype='float64')  # Ensure current_weights is float64

#     # Define test input
#     combined_categories = [('category', 'A')]
#     target_proportion = 0.75  # 75%

#     # Call the function
#     updated_weights = update_weights_for_combined_category(
#         df, combined_categories, target_proportion, current_weights)

#     # Ensure expected_weights is float64
#     expected_weights = pd.Series([1.5, 1, 1.5, 1], dtype='float64')
#     # Debugging: Print statements to check values
#     print("Updated weights:", updated_weights)
#     print("Expected weights:", expected_weights),      
#     # Assert
#     pd.testing.assert_series_equal(updated_weights, expected_weights)

# # Call your test function
# test_basic_functionality()


In [143]:
# import matplotlib.pyplot as plt

# # Now, let's plot the histogram of the 'Weight' column using the same style as the image provided
# plt.figure(figsize=(10, 6))  # Set the size of the plot
# plt.hist(df_modified['Weight'], bins=40, color='darkseagreen', edgecolor='black')  # Specify the number of bins and color

# # Aesthetics for the plot to match the style of the provided image
# plt.title('Distributia ponderilor', fontsize=10, fontweight='bold')
# plt.xlabel('Pondere', fontsize=12)
# plt.ylabel('Frecvență', fontsize=12)
# plt.xticks(fontsize=10)
# plt.yticks(fontsize=10)
# plt.grid(axis='y', linestyle='--', alpha=0.4)

# # Show the plot
# plt.show()

In [144]:
# df_modified['Weight'].describe()

# Test question selection

In [145]:
# # Selectam fisierul
# filename = input("Introduceti denumirea fisierului care contine textul intrebarilor: ")
# filename1 = f"{filename}.xls"

# df_questions = pd.read_excel(filename1)

# # Method 1: Using loc
# df_filtered = df_questions.loc[df_questions['Is Topic'] != False]
# # Dropping the first two rows of df_filtered
# df_filtered = df_filtered.iloc[2:]
# # Dropping rows where 'Question Index' column has NA/null values
# df_filtered = df_filtered.dropna(subset=['Question Index'])
# # Selecting only 'Question Index' and 'Text' columns
# df_filtered = df_filtered[['Question Index', 'Text']]
# # Replacing <b> and </b> with an empty string in the 'Text' column
# df_filtered['Text'] = df_filtered['Text'].replace('<b>|</b>', '', regex=True)
# # Extracting column names that start with "Q" and "T_Q"
# filtered_columns = [col for col in df_original.columns if col.startswith('Q_') or col.startswith('T_Q')]

# # Creating a new DataFrame with these column names as values in a single column
# df_columns = pd.DataFrame({'column_names': filtered_columns})

# # Create a new DataFrame for modified data with T_Q_digit part as the header of each section
# df_columns_new = pd.DataFrame(columns=df_columns.columns)

# # Initialize last_group as None to indicate no group has been encountered yet
# last_group = None

# for index, row in df_columns.iterrows():
#     # Extract the group number from the 'T_Q' pattern
#     match = re.match(r'T_Q_(\d+)_\d+', str(row['column_names']))
#     if match:
#         current_group = int(match.group(1))

#         # Insert a row with the T_Q_digit part if it's the first group or if the group number changes
#         if last_group is None or current_group != last_group:
#             df_columns_new = df_columns_new.append(pd.Series({'column_names': f'T_Q_{current_group}'}), ignore_index=True)

#         # Update the last seen group number
#         last_group = current_group

#     # Append the current row
#     df_columns_new = df_columns_new.append(row)

# # Reset index for the new DataFrame
# df_columns_new.reset_index(drop=True, inplace=True)

# def filter_specific_pattern(dataframe, pattern_to_keep, general_pattern):
#     # Function to check if the entry matches the general pattern and ends with the specific pattern
#     def entry_matches(entry):
#         return re.match(general_pattern, entry) and entry.endswith(pattern_to_keep)

#     # Apply the filter to the DataFrame
#     filtered_dataframe = dataframe[
#         dataframe['column_names'].apply(lambda x: not re.match(general_pattern, x) or entry_matches(x))
#     ]

#     return filtered_dataframe

# # The specific pattern we want to keep
# specific_ending = '_O1'

# # The general pattern that the questions must match (e.g., 'Q_number_Osomething')
# question_pattern = r'Q_\d+_O\d+'

# # Apply the filter function to the DataFrame
# df_columns_new = filter_specific_pattern(df_columns_new, specific_ending, question_pattern)

# # Step 1: Remove '_O1' from the end of strings in 'column_names'
# df_columns_new['column_names'] = df_columns_new['column_names'].str.replace('_O1', '', regex=False)

# # Step 2: Drop rows where 'column_names' ends with 'S'
# df_columns_new = df_columns_new[~df_columns_new['column_names'].str.endswith('S')]

# # Truncate df_filtered to match the length of df_columns_new
# df_filtered = df_filtered.iloc[:len(df_columns_new)]

# # Assign values from 'column_names' in df_columns_new to 'q_index' in df_filtered
# df_filtered['q_index'] = df_columns_new['column_names'].values

# # Reorder columns and drop 'Question index'
# df_filtered = df_filtered[['q_index', 'Text']].copy()


In [146]:
unique_values_list

['Russian speakers 15-29 years.',
 'Russian speakers 30-39 years.',
 'Romanian speakers 15-29 years.']